# Tutorial 5 -- Human Detection 
This tutorial will show you how to detect human with the robot. We will use multi-thread to capture the color images and depth images. Then the ssd mobilenet v2 nerual network structure is used for human detection. To enable real-time performance, the tensorRT deep learning framework is used. More information can be found in the nvidia's jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot

### Step 1 load the pre-trained object detection model

Before running the program, we need to download the pre-trained detection model and put it in the current folder. The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of these two values will be captured by traitlets
    color_value = traitlets.Any()
    depth_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        self.stop_flag = False
        
        #start capture the first depth and color image
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        depth_frame = frames.get_depth_frame()           
        self.depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   
        
        self.warning_flag = 0
        self.min_distance = 0

    def _capture_frames(self):
        while(self.stop_flag==False):
            frames = self.pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            image = np.asanyarray(color_frame.get_data())
            self.color_value = image

            depth_frame = frames.get_depth_frame()              
            self.depth_image = np.asanyarray(depth_frame.get_data())
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap
            
            depths = self.depth_image[self.depth_image!=0]
            if depths.size == 0:
                self.min_distance = 0
            else:
                self.min_distance = depths.min()
            self.warning_flag = (self.min_distance < 200)
    
    def start(self):
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        self.stop_flag = True
        if hasattr(self, 'thread'):
            self.thread.join()        
        if self.pipeline_started:
            self.pipeline.stop()

def bgr8_to_jpeg(value,quality=75):
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()


In [3]:
camera.start() # start capturing the data

### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [4]:
from mochris import RangeSensors
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

obstacle_slider = widgets.IntSlider(description='Obstacle', min=40, max=4000, orientation='horizontal')

range_sensors = RangeSensors(front=True)

def update(change):
    front_range = change['new']    
    obstacle_slider.value = front_range

range_sensors.observe(update, names='front_value')

display(obstacle_slider)

IntSlider(value=40, description='Obstacle', max=4000, min=40)

In [7]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
from random import random

width = 640
height = 480

# Creates a widgit to display the image from camera on
image_widget = widgets.Image(format='jpeg', width=300, height=300)

# Creates the label widget to select tracked object
label_widget = widgets.IntText(value=27, description='tracked label')

# Creates a label widget to show the current distance from tracked object
label_distance = widgets.IntText(value=1, description='distance')

# Creates a label widget to show the minimum distance from tracked object
label_min_distance = widgets.IntText(value=0, description='minimum distance')

# Creates the drop down menu to select the different behaviors.
# Default is love.
mode_widget = widgets.Dropdown(
    options=['aggressive', 'fear', 'love', 'curious'],
    value='love',
    description='Mode: ',
    disabled=False,
)
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget,
    label_distance,
    obstacle_slider,
    mode_widget
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

# Defines the current state the robot is in for the love behavior. 
love_flag = 0

# Defines when the robot should be stopped. 
stop_flag = 0

# Defines the direction the robot should turn.
# 0 = right
# 1 = left
left_flag = 1

# The love behavior logic. When love is selected, this function is called
# and the robot performs a left,right motion going forward.
def love():
    global love_flag
    if love_flag == 0:
        robot.left(0.5)
    elif love_flag == 1:
        robot.forward(0.3)
    elif love_flag == 2:
        robot.right(0.5)
    elif love_flag == 3:
        robot.forward(0.3)
    love_flag = (love_flag + 1) % 4

curious_flag = 0

# The curious behavior logic. 
# param direction: The direction the robot should move, value should be 'left' or 'right'
def curious(direction):
    global curious_flag
    if curious_flag <= 10:
        if direction == 'left':
            robot.forward_left(0.5)
        elif direction == 'right':
            robot.forward_right(0.5)
        else:
            robot.forward(0.5)
    else:
        robot.backward(0.3)
    curious_flag = (curious_flag + 1) % 20

def processing(change):
    image = change['new']
    depth = camera.depth_image
    global stop_flag
    global left_flag
    
    # The number of objects on the left hand side of the screen     
    left_count = 0
    
    # The number of objects in the middle of the screen     
    center_count = 0
    
    # The number of objects on the right hand side of the screen    
    right_count = 0
    size_largest = 0
     
    imgsized= cv2.resize(image,(300,300))
    
    # compute all detected objects
    detections = model(imgsized)
    
    # Gets all the detected object that match the object we are trying to detect.      
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    for det in matching_detections:
        bbox = det['bbox']
        
        x1 = int(width * bbox[0])
        x2 = int(width * bbox[2])
        y1 = int(width * bbox[1])
        y2 = int(width * bbox[3])
        depths = depth[y1:y2, x1:x2]
        depths = depths[depths != 0]
        distance = np.nanmean(depths)
        
        # Draws a bounding box around objects that are not the tracked one.         
        if det['confidence'] < 0:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            continue
        else:
            # Draw an image in red around the detected image             
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
      
        # Counts the number of objects on the left, right and center of the image.         
        if (bbox[0]+bbox[2])/2 < 2/5.0:
            left_count = left_count + 1
        elif (bbox[0]+bbox[2])/2 > 3/5.0:
            right_count = right_count + 1
        else:
            center_count = center_count + 1
        label_distance.value = distance
        size = (bbox[2] - bbox[0]) * (bbox[3] - bbox[2])
        if size > size_largest:
            size_largest = size
        
    label_min_distance.value = camera.min_distance
    
    # Wander behavior     
    if love_flag:
        love()
        if len(matching_detections) == 0:
            stop_flag = 3
    elif len(matching_detections) == 0:
        if random() < 0.5:
            if stop_flag:
                stop_flag = max(stop_flag - 1, 0)
                robot.stop()
            else:
                if left_flag:
                    robot.left(0.5)
                else:
                    robot.right(0.5)
        else:
            robot.stop()
    else:
        if left_count > right_count and left_count > center_count:
            left_flag = 1
        if right_count > left_count and right_count > center_count:
            left_flag = 0
        stop_flag = 0
        
        # Handles the fear behavior         
        if mode_widget.value == 'fear':
            if distance < 1800:
                robot.backward(max((3000-distance)/3000*2, 0.3))
            else:
                stop_flag = 3
                robot.stop()
         
        # Handles the aggression behavior         
        elif mode_widget.value == 'aggressive':
            if distance < 1200:
                robot.stop()
            elif left_count > right_count and left_count > center_count:
                left_flag = 1
                robot.forward_left(max((5000-distance)/2500*2+0.3, 0.3))
            elif right_count > left_count and right_count > center_count:
                left_flag = 0
                robot.forward_right(max((5000-distance)/2500*2+0.3, 0.3))
            else:
                robot.forward(max((2000-distance)/2000*2, 0.3))
                
        # Handles the love behavior                 
        elif mode_widget.value == 'love':
            if distance < 900:
                robot.stop()
            elif left_count > right_count:
                stop_flag = 3
                robot.left(0.5)
            elif right_count > left_count:
                stop_flag = 3
                robot.right(0.5)
            else:
                love()
                
        # Handles the curious behavior                 
        elif mode_widget.value == 'curious':
            if distance < 900:
                robot.stop()
            elif left_count > right_count and left_count > center_count:
                stop_flag = 3
                robot.left(0.5)
            elif right_count > left_count and right_count > center_count:
                stop_flag = 3
                robot.right(0.5)
            else:
                curious('center')
    

    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
processing({'new': camera.color_value})
camera.observe(processing, names=['color_value'])

The following code can be used to stop the capturing of the image and the moving of the robot

In [8]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()

In [ ]:
camera.stop()

## Tasks

### 1. Please try to calculate the distance between the detected human and the robot using the depth image. (Note: You can refer to tutorial 2 to get the depth information for a specific point)
### 2. Please try to add collision avoidance function into this program to protect the robot. (Note: You can refer to tutorial 4 for the collision avoidance)
### 3. Think about how to control the robot to move towards the detected human